In [8]:
import pandas as pd
from datetime import time
from database.db_utils import get_connection
pd.set_option('display.max_columns', None)

In [9]:
# connect to the database
engine = get_connection()

# load the tables into pd.Dataframes
store_status_df: pd.DataFrame = pd.read_sql_table('store_status', con=engine)
time_zone_df: pd.DataFrame = pd.read_sql_table('time_zone', con=engine)
menu_hours_df: pd.DataFrame = pd.read_sql_table('menu_hours', con=engine)
reports_df: pd.DataFrame = pd.read_sql_table('reports', con=engine)

In [ ]:
merged_df = pd.merge(store_status_df, menu_hours_df, on='store_id', how='left')
merged_df = pd.merge(merged_df, time_zone_df, on='store_id', how='left')

merged_df['timestamp_utc'] = pd.to_datetime(merged_df['timestamp_utc'])
merged_df['timestamp_utc'] = merged_df['timestamp_utc'].dt.tz_localize('UTC')

merged_df['timezone_str'] = merged_df['timezone_str'].fillna('America/Chicago')
merged_df['timestamp_local'] = merged_df\
    .apply(lambda row: row['timestamp_utc'].tz_convert(row['timezone_str']), axis=1)
grouped = merged_df.groupby(by=['store_id'])\
    .agg(min_value=('timestamp_local', 'min'), max_value=('timestamp_local', 'max'))

print(grouped)